# Easily create an AIConfig from existing Openai code
1. Basic usage
2. Load existing aiconfig and continue
3. Capture function calling
4. Use Client API
5. Save to existing AIConfig (no JSON)

## 1. Basic usage

In [1]:
# Install AIConfig package
!pip3 install python-aiconfig

# Create ~/.env file with this line: export OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from aiconfig.ChatCompletion import get_completion_create_wrapped_openai

output_path = "my-first-aiconfig.json"

# replace openai import with this
openai = get_completion_create_wrapped_openai(
    output_aiconfig_ref=output_path,
)

def run_my_existing_openai_app(user_message: str):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": False,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
    print("Chat Completion Response: ")
    print(type(response))

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Run your code as usual
run_my_existing_openai_app("Tell me a joke about apples")

[INFO] 2024-01-04 22:11:33,787 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo': 'gpt-3.5-turbo', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Tell me a joke about apples', 'role': 'user'}]}, 'prompt_name': 'prompt', 'params': None} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:33,789 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'prompt', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Tell me a joke about apples', 'role': 'user'}]}, 'parameters': {}, 'kwargs': {}} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:33,790 callback.py:140: Callback called. event
: name='on_serialize_complete' file='aiconfig.default_parsers.openai' data={'result': [Prompt(name='prompt', input='Tel

Chat Completion Response: 
<class 'openai.types.chat.chat_completion.ChatCompletion'>


In [4]:
import json

print("my-first-aiconfig.json now represents your existing app:\n")
result = json.load(open(output_path))
print(
    json.dumps(result, indent=2)
)

my-first-aiconfig.json now represents your existing app:

{
  "name": "",
  "schema_version": "latest",
  "metadata": {
    "parameters": {},
    "models": {}
  },
  "description": "",
  "prompts": [
    {
      "name": "prompt_0",
      "input": "Tell me a joke about apples",
      "metadata": {
        "model": {
          "name": "gpt-3.5-turbo",
          "settings": {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": false
          }
        },
        "parameters": {},
        "remember_chat_context": true
      },
      "outputs": [
        {
          "output_type": "execute_result",
          "execution_count": 0,
          "data": {
            "content": "Why don't apples ever get lonely? \n\nBecause they all have a lot of core friends!",
            "role": "assistant"
          },
          "metadata": {
            "id": "chatcmpl-8dUuLsPM30SfOKklmGvkKKKj2cx0i",
            "c

In [5]:
import json

print("Zoom in on the prompt:\n")
print(
    json.dumps(result["prompts"][0], indent=2)
)

Zoom in on the prompt:

{
  "name": "prompt_0",
  "input": "Tell me a joke about apples",
  "metadata": {
    "model": {
      "name": "gpt-3.5-turbo",
      "settings": {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": false
      }
    },
    "parameters": {},
    "remember_chat_context": true
  },
  "outputs": [
    {
      "output_type": "execute_result",
      "execution_count": 0,
      "data": {
        "content": "Why don't apples ever get lonely? \n\nBecause they all have a lot of core friends!",
        "role": "assistant"
      },
      "metadata": {
        "id": "chatcmpl-8dUuLsPM30SfOKklmGvkKKKj2cx0i",
        "created": 1704424293,
        "model": "gpt-3.5-turbo-0613",
        "object": "chat.completion",
        "usage": {
          "completion_tokens": 19,
          "prompt_tokens": 13,
          "total_tokens": 32
        },
        "finish_reason": "stop"
      }
    }
  ]
}


## 2. Continue from existing aiconfig file

In [6]:
# Run your code as usual
run_my_existing_openai_app("Tell a joke about apples in Shakespearian English.")

[INFO] 2024-01-04 22:11:34,484 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo': 'gpt-3.5-turbo', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Tell a joke about apples in Shakespearian English.', 'role': 'user'}]}, 'prompt_name': 'prompt', 'params': None} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:34,485 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'prompt', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Tell a joke about apples in Shakespearian English.', 'role': 'user'}]}, 'parameters': {}, 'kwargs': {}} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:34,486 callback.py:140: Callback called. event
: name='on_serialize_complete' file='aiconfig.default_parsers.openai' dat

Chat Completion Response: 
<class 'openai.types.chat.chat_completion.ChatCompletion'>


In [7]:
print("my-first-aiconfig.json has your existing prompts:\n")

prompts = json.load(open(output_path))["prompts"]
print(
    json.dumps(prompts[0], indent=2)
)

my-first-aiconfig.json has your existing prompts:

{
  "name": "prompt_0",
  "input": "Tell me a joke about apples",
  "metadata": {
    "model": {
      "name": "gpt-3.5-turbo",
      "settings": {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": false
      }
    },
    "parameters": {},
    "remember_chat_context": true
  },
  "outputs": [
    {
      "output_type": "execute_result",
      "execution_count": 0,
      "data": {
        "content": "Why don't apples ever get lonely? \n\nBecause they all have a lot of core friends!",
        "role": "assistant"
      },
      "metadata": {
        "id": "chatcmpl-8dUuLsPM30SfOKklmGvkKKKj2cx0i",
        "created": 1704424293,
        "model": "gpt-3.5-turbo-0613",
        "object": "chat.completion",
        "usage": {
          "completion_tokens": 19,
          "prompt_tokens": 13,
          "total_tokens": 32
        },
        "finish_reason": "stop"
      }

In [8]:
print("And your new prompt:\n")
print(
    json.dumps(prompts[1], indent=2)
)

And your new prompt:

{
  "name": "prompt_1",
  "input": "Tell a joke about apples in Shakespearian English.",
  "metadata": {
    "model": {
      "name": "gpt-3.5-turbo",
      "settings": {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": false
      }
    },
    "parameters": {},
    "remember_chat_context": true
  },
  "outputs": [
    {
      "output_type": "execute_result",
      "execution_count": 0,
      "data": {
        "content": "Why did the apple turneth redeth?\n\nBecause 'twas afraid of becoming Juliet's fair roseth!",
        "role": "assistant"
      },
      "metadata": {
        "id": "chatcmpl-8dUuMhp3JB7LOhtedqjeYKVrw5fkY",
        "created": 1704424294,
        "model": "gpt-3.5-turbo-0613",
        "object": "chat.completion",
        "usage": {
          "completion_tokens": 23,
          "prompt_tokens": 19,
          "total_tokens": 42
        },
        "finish_reason": "stop"
    

# 3. Capture function calling

### My existing app using function calling

In [9]:
# Function Call Capture


from typing import Any

output_path = "my-function-calling-aiconfig.json"

# replace openai import with this
openai = get_completion_create_wrapped_openai(
    output_aiconfig_ref=output_path,
)


def get_current_weather(location: str, unit: str) -> dict[str, Any]:
    return { "temperature": 22, "unit": "celsius", "description": "Sunny" }


def run_my_existing_weather_function_calling_app():
    completion_params = {
        "model": "gpt-3.5-turbo-0613",
        "messages": [{"role": "user", "content": "What is the weather like in Boston?"}],
        "functions": [
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
    }

    response = openai.chat.completions.create(**completion_params) 

    function_call_response = get_current_weather(location="Boston", unit="celsius")
    print(response)

    completion_params = {
      "model": "gpt-3.5-turbo-0613",
      "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
        {"role": "assistant", "content": 'null', "function_call": {
              "name": "get_current_weather",
              "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
            }},
        {"role": "function", "name": "get_current_weather", "content": str(function_call_response)}

      ],
      "functions": [
        {
          "name": "get_current_weather",
          "description": "Get the current weather in a given location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
              },
              "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"]
              }
            },
            "required": ["location"]
          }
        }
      ]
    }

    response = openai.chat.completions.create(**completion_params) 
    print(type(response))

In [10]:
# Run your code as usual
run_my_existing_weather_function_calling_app()

[INFO] 2024-01-04 22:11:35,410 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo-0613': 'gpt-3.5-turbo-0613', 'data': {'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'What is the weather like in Boston?'}], 'functions': [{'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['location']}}]}, 'prompt_name': 'prompt', 'params': None} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:35,411 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'prompt', 'data': {'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'What is the weather like in Boston?'}], 'func

ChatCompletion(id='chatcmpl-8dUuMTRR32mrFFzNzhZBIbOIAnWPL', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None), logprobs=None)], created=1704424294, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=82, total_tokens=99))


[INFO] 2024-01-04 22:11:35,778 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo-0613': 'gpt-3.5-turbo-0613', 'data': {'model': 'gpt-3.5-turbo-0613', 'messages': [{'role': 'user', 'content': 'What is the weather like in Boston?'}, {'role': 'assistant', 'content': 'null', 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Boston, MA"\n}'}}, {'role': 'function', 'name': 'get_current_weather', 'content': "{'temperature': 22, 'unit': 'celsius', 'description': 'Sunny'}"}], 'functions': [{'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['location']}}]}, 'prompt_name': 'prompt', 'params': None} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:3

<class 'openai.types.chat.chat_completion.ChatCompletion'>


In [11]:
print("Inspect my-function-calling-aiconfig.json:\n")

prompts = json.load(open(output_path))["prompts"]
print(
    json.dumps(prompts[0], indent=2)
)

Inspect my-function-calling-aiconfig.json:

{
  "name": "prompt_0",
  "input": "What is the weather like in Boston?",
  "metadata": {
    "model": {
      "name": "gpt-3.5-turbo-0613",
      "settings": {
        "model": "gpt-3.5-turbo-0613",
        "functions": [
          {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
              "type": "object",
              "properties": {
                "location": {
                  "type": "string",
                  "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                  "type": "string",
                  "enum": [
                    "celsius",
                    "fahrenheit"
                  ]
                }
              },
              "required": [
                "location"
              ]
            }
          }
        ]
      }
    },
    "parameters": {}

## 4. Use Client API

In [12]:
from aiconfig.ChatCompletion import get_completion_create_wrapped_openai_client


output_path = "my-aiconfig-from-Client-API.json"

client = get_completion_create_wrapped_openai_client(output_path)

def run_my_existing_client_api_app():
    completion_params = {
                "model": "gpt-3.5-turbo",
                "top_p": 1,
                "max_tokens": 3000,
                "temperature": 1,
                "stream": False,
                "messages": [
                    {
                        "content": "Compare and contrast bananas and cucumbers.",
                        "role": "user",
                    }
                ],
            }
    response = client.chat.completions.create(**completion_params)
    print(type(response))

    


In [13]:
# Run your code as usual
run_my_existing_client_api_app()

[INFO] 2024-01-04 22:11:43,565 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo': 'gpt-3.5-turbo', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Compare and contrast bananas and cucumbers.', 'role': 'user'}]}, 'prompt_name': 'prompt', 'params': None} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:43,566 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'prompt', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Compare and contrast bananas and cucumbers.', 'role': 'user'}]}, 'parameters': {}, 'kwargs': {}} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:43,568 callback.py:140: Callback called. event
: name='on_serialize_complete' file='aiconfig.default_parsers.openai' data={'result': [

<class 'openai.types.chat.chat_completion.ChatCompletion'>


In [14]:
import json

print(f"Inspect {output_path}:\n")

prompts = json.load(open(output_path))["prompts"]
print(
    json.dumps(prompts[0], indent=2)
)

Inspect my-aiconfig-from-Client-API.json:

{
  "name": "prompt_0",
  "input": "Compare and contrast bananas and cucumbers.",
  "metadata": {
    "model": {
      "name": "gpt-3.5-turbo",
      "settings": {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": false
      }
    },
    "parameters": {},
    "remember_chat_context": true
  },
  "outputs": [
    {
      "output_type": "execute_result",
      "execution_count": 0,
      "data": {
        "content": "Bananas and cucumbers are both popular fruits, but they differ in various aspects. Let's compare and contrast them:\n\n1. Appearance: Bananas are elongated, curved fruits with a yellow color when ripe, while cucumbers are cylindrical with a green color throughout their ripening process. \n\n2. Taste: Bananas have a sweet taste with a creamy texture when ripe, while cucumbers have a crisp texture and a refreshing, mild flavor.\n\n3. Nutritional Value: Banana

## 5. Save to existing AIConfig (no JSON)

In [15]:
from aiconfig.ChatCompletion import get_completion_create_wrapped_openai
from aiconfig.Config import AIConfigRuntime

existing_aiconfig = AIConfigRuntime.create()

# replace openai import with this
openai = get_completion_create_wrapped_openai(
    output_aiconfig_ref=existing_aiconfig,
)

def run_my_existing_openai_app(user_message: str):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": False,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
    print("Chat Completion Response: ")
    print(type(response))

In [16]:
run_my_existing_openai_app("Are tomatoes fruits?")

[INFO] 2024-01-04 22:11:44,890 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo': 'gpt-3.5-turbo', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Are tomatoes fruits?', 'role': 'user'}]}, 'prompt_name': 'prompt', 'params': None} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:44,891 callback.py:140: Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'prompt', 'data': {'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False, 'messages': [{'content': 'Are tomatoes fruits?', 'role': 'user'}]}, 'parameters': {}, 'kwargs': {}} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:11:44,891 callback.py:140: Callback called. event
: name='on_serialize_complete' file='aiconfig.default_parsers.openai' data={'result': [Prompt(name='prompt', input='Are tomatoes frui

Chat Completion Response: 
<class 'openai.types.chat.chat_completion.ChatCompletion'>


In [17]:
await existing_aiconfig.run("prompt_0")

print("Result:")
print(existing_aiconfig.get_output_text("prompt_0"))

[INFO] 2024-01-04 22:13:00,362 callback.py:140: Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'prompt_0', 'params': None, 'options': None, 'kwargs': {}} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:13:00,364 callback.py:140: Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='prompt_0', input='Are tomatoes fruits?', metadata=PromptMetadata(model=ModelMetadata(name='gpt-3.5-turbo', settings={'model': 'gpt-3.5-turbo', 'top_p': 1, 'max_tokens': 3000, 'temperature': 1, 'stream': False}), tags=None, parameters={}, remember_chat_context=True), outputs=[]), 'options': None, 'parameters': {}} ts_ns=1704424292516630000
[INFO] 2024-01-04 22:13:00,365 callback.py:140: Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='prompt_0', input='Are tomatoes fruits?', metadata=PromptMetadata(model=ModelMetadata(name='gpt-3.5-turbo', 

Result:
Yes, tomatoes are technically considered fruits. They develop from the ovary of a flowering plant and contain seeds. However, they are commonly referred to as vegetables in culinary contexts due to their savory taste and culinary usage.
